# 部署至批次端點

想像一下，健康醫生每天都會進行病患測量，並將每個病患的詳細資料儲存在個別的檔案中。 然後，糖尿病預測模型可以用來將當天的所有病患資料當成批次處理，產生預測，這些預測將在早上等候，讓臨床可以追蹤預測為有糖尿病風險的病患。 透過 Azure Machine Learning，您可以藉由建立批次端點來完成這項作業;這就是您將在本練習中實作的內容。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 註冊模型

批次部署只能部署在工作區中註冊的模型。 您將註冊儲存在本機 `model` 資料夾中的 MLflow 模型。 

In [ ]:
## 建立批次端點

批次端點是 HTTPS 端點，應用程式可以呼叫以觸發批次評分作業。 批次端點名稱在 Azure 區域內必須是唯一的。 您將使用 函 `datetime` 式，根據目前的日期和時間產生唯一的名稱。 

In [ ]:
若要使用 `BatchEndpoint` 類別建立端點，您必須指定名稱和選擇性的描述。 建立端點之後，您會將模型部署至端點。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到端點建立後再繼續！ 綠色通知應該會出現在 Studio 中。 </p>

## 建立部署

部署是託管執行實際推斷模型所需的一組資源。 我們將使用 `BatchDeployment` 類別為端點建立部署。 

因為您要部署 MLflow 模型，所以不需要評分腳本或定義環境。 Azure Machine Learning 會自動為您建立這些資產。 `MLmodel`資料夾中的 `model` 檔案是用來瞭解模型的預期輸入和輸出。

您將部署具有下列參數的模型：

- `name`：部署的名稱。
- `description`：選擇性描述，以進一步厘清部署所代表的內容。
- `endpoint_name`：應該部署模型之先前建立端點的名稱。
- `model`：已註冊模型的名稱。
- `compute`：叫用已部署的模型來產生預測時要使用的計算。
- `instance_count`：用於產生預測的計算節點計數。
- `max_concurrency_per_instance`：每個計算節點執行的平行評分腳本數目上限。
- `mini_batch_size`：每個評分腳本執行所傳遞的檔案數目。
- `output_action`：每個新的預測都會附加為輸出檔案的新資料列。
- `output_file_name`：將附加預測的檔案。
- `retry_settings`：迷你批次的設定失敗。
- `logging_level`：記錄詳細資訊層級。 允許的值為：`warning`、`info` 和 `debug`。 

執行下列資料格將會設定及建立部署。

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到部署完成，再繼續！ 綠色通知應該會出現在 Studio 中。 </p>

您可以將多個模型部署到端點。 您可以設定預設部署，以指定呼叫批次端點時應該使用哪一個模型。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要！ 等到預設部署設定好再繼續！ 綠色通知應該會出現在 Studio 中。 </p>

## 準備批次預測的資料

在 `data` 資料夾中，您會找到具有未標記資料的 CSV 檔案。 您將建立一個資料資產，指向資料夾中的檔案 `data` ，您將用來做為批次作業的輸入。

In [ ]:
## 提交工作

現在您已將模型部署至批次端點，並擁有未標記的資料資產，您現在可以叫用端點來產生未標記資料的預測。

首先，您將藉由參考已註冊的資料資產來定義輸入。 然後，您將叫用端點，以提交管線作業。 您可以使用作業 URL 在 Studio 中監視它。 作業會包含子作業，代表 (產生的) 評分腳本的執行，以取得預測。

## 取得結果

當叫用批次端點的管線作業完成時，您可以檢視結果。 所有預測都會收集在 `predictions.csv` 儲存在預設資料存放區的檔案中。 您可以藉由執行下列資料格來下載檔案，並將資料視覺化。 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score